# Installing Libraries

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install spacy
!sudo apt update

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [901 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get

In [ ]:
!pip install pytesseract
!apt-get install poppler-utils
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).

**Note:** Restart Runtime After Installation

# Importing Packages

In [ ]:
import cv2 
import matplotlib.pyplot as plt
import pytesseract
import shutil
import os
import random
import nltk
import numpy as np
import re
try:
 from PIL import Image
except ImportError:
 import Image

# Downloading Packages

In [ ]:
!python3 -m spacy download en_core_web_sm

2022-08-04 11:25:37.672971: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 6.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pre Processing Image

In [ ]:
def ZoomImage(img, zoom_factor=2):
    return cv2.resize(img, None, fx= zoom_factor, fy= zoom_factor, interpolation= cv2.INTER_LINEAR)

def GrayScaleImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255*(gray < 128).astype(np.uint8) 
    return gray

def InvertImage(img):
    return cv2.bitwise_not(img)

def CropImage(img):
    # Find all non-zero points (text)
    coords = cv2.findNonZero(img) 
    # Find minimum spanning bounding box
    x, y, w, h = cv2.boundingRect(coords) 
    # Crop the image - note we do this on the original image
    img = img[y:y+h, x:x+w] 
    return img

# Creating Bounding Box In Image

In [ ]:
def CreateBoundingBox(img):
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img) 
    for b in boxes.splitlines():
        b = b.split(' ')
        img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
    return img

# Extracting Text From Image

In [ ]:
def ExtractText(img) : 
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(img, config=custom_config)
    return text

# Extracting Name

Pre Processing For Name

In [ ]:
def text_preprocessing(text):
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  newSentence = ""
  for word in text:
      if (word in punctuations):
          newSentence = newSentence + " "
      else: 
          newSentence = newSentence + word

  return newSentence

def remove_white_spaces(text):
  token_words = word_tokenize(text)
  new_sentence = []
  for word in token_words:
    new_sentence.append(word) 
    new_sentence.append(" ") 
  return "".join(new_sentence)

def removing_extra(text):
  new_sentence = ""
  result = ""
  token_sentence = sent_tokenize(text)
  if "applying for" in token_sentence[0].lower() or "applied for" in token_sentence[0].lower() or "post applied" in token_sentence[0].lower():
    sentences = token_sentence[0].split("\n")
    for sent in sentences:
      if "applying for" in sent.lower() or "applied for" in sent.lower():
        continue
      else:
        new_sentence = new_sentence + sent + "\n"

    i = 0
    for sentence in token_sentence:
      if i == 0:
        result = result + new_sentence + "\n"
      else:
        result = result + sentence  + "\n"
      i += 1
    return result 
  else:
    return text

In [ ]:
# import spacy
# from spacy.matcher import Matcher

# nlp = spacy.load('en_core_web_sm')

# #words to be avoided in extraction
# avoid_words = ['resume','résumé','cv', 'curriculum','marketing', 'curriculum vitae','vitae', 'curriculum vita',
#                  'vita', 'name', 'pg', 'india', 'curriculam vitae', 'business development', 'cast', 'coc,','uae',
#                  'address', 'email', 'email address', 'sales', 'u.p', 'product development', 'personal statement',
#                  'full Name','street address', 'mobile', 'phone', 'number', 'phone number', 'mobile number', 'product',
#                  'agarwal', 'inter', 'secondary', 'steels', 'element', 'engineering','system', 'school', 'operating',
#                  'acknowledgement', 'assimilated', 'collaborated', 'collaboration','collaborations', 'india', 'isf,',
#                  'professional summary', 'professional', 'title', 'job title', 'job' 'technological', 'high', 'college',
#                  'top', 'device', 'launch', 'ams', 'crm', 'P Technological', 'project engineer', 'cgpa', 'gpa',
#                  'personal profile', 'years', 'lv', 'career', 'career profile','lv', 'curriculum viate', 'civil',
#                  'post applied', 'applied', 'post', 'for', 'electrical engineer', 'ps', 'district', 'ilets',
#                  'career objective.', 'it', 'carrier objective', 'electrical project handling', 'mig,', 'hvac',
#                  'mdu', 'linkedin', 'facebook', 'behance', 'youtube', 'github', 'in', 'portfolio','total experience',
#                  'pbd','working','xcx','wwd','mba','dtdc','cf ril', 'fi', 'ef', 'dd', 'project management',
#                  'engineer', 'apj', 'course university', 'university', 'ltd.', 'educational', 'n.y.', 's.s.',
#                  'information technology', 'uk', 'usa', 'uae','pakistan','china','canada','saudia','arabia',
#                  'educational background', 'details', 'personal details', 'clout', 'cloud', 'inditex', 'vy.', 'instrument',
#                  'instrument super', 'bca', 'u.p.', 'qa', 'gis', 'gre', 'aicte', 'professional and', 'education background',
#                  'tx', 'pvc', 'acadmic qualification', 'apply for merchandiser', 'education background','bechelor',
#                  'beardsell pvt.', 'class', 'fi ef', 'uae,', 'of', 'b.s.', 'project', 'applied for','applying for',
#                  'contributed', 'participated', 'participations', 'partnered', 'partnerships', 'volunteer work',
#                  'volunteer', 'side activities', 'projects', 'education', 'experience', 'work', 'work experience',
#                  'skills', 'interests', 'biography', 'bio', 'about me', 'about', 'info', 'contact', 'contact info',
#                  'reference', 'reference will be established upon request', 'certificates', 'additional certificates',
#                  'courses', 'student', 'online courses', 'short courses', 'languages', 'internships', 'organizations',
#                  'organization', 'matriculation', 'intermediate', 'pre-engineering', 'bachelors', 'masters',
#                  'bscs', 'bachelor of science', 'btech', 'btech', 'bachelor of technology', 'bs-cs,', 'sslc',
#                  'bse', 'bachelor of commerce', 'bcom', 'b.com', 'information', 'personal information', 'curriculam',
#                  'info','achievements', 'personal projects', 'personal skills', 'summary', 'courses and certificates:',
#                  'profile', 'hobbies & interests', 'hobbies and interests', 'hobbies', 'career objective',
#                  'objective', 'other skills', 'academic', 'academics', 'academic qualification', 'references', 'viate',
#                  'personal', 'cnic', 'dob', 'date of birth', 'work history','computer skills', 'referenc', 'mechanical',
#                  'economics and finance', 'top skills', 'personal email', 'contact no', 'contact no.', 'electrical',
#                  'cnic no', 'cnic no.', 'religion', 'domicile', 'a levels', 'o levels', 'working links', 'e mail',
#                  'links', 'working experience', 'awards and achievements', 'awards', 'soft skills', 'diploma holder',
#                  'volunteer experience', 'conferences and courses', 'conferences', 'educational qualification',
#                  'educational qualifications', 'value addition', 'declaration', 'brief employment history',
#                  'extra activities', 'extra', 'skills and responsibilities', 'duties and responsibilities',
#                  'skills and abilities', 'area of expertise', 'technical skills', 'current address', 'street',
#                  'permanent address', 'passport no', 'expiry date', 'career summary', 'email id', 'phone no',
#                  'contact no', 'contact no.', 'phone no.', 'education qualification', 'python', 'js6', 'ansys',
#                  'critical', 'conditions', 'handling', 'a', 'b', 'c', 'd', 'e', 'f','g','h','i','j','k','l','m',
#                  'n','o','p','q','r','s','t','u','v','w','x','y','z'
#                  ]

# #creating an instance of matcher for real words with nlp.vocab
# matcher = Matcher(nlp.vocab)

# #extracts names
# def extract_name(content):

#     #pre-processing to remove any faulty punctuation
#     new_content = text_preprocessing(content)
    
#     #data annotation
#     nlp_text = nlp(new_content)
    
#     #patterns for names
#     patterns = [
#                [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],
#                [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],
#                [{'POS': 'PROPN'}, {'POS': 'PROPN'}],
#                [{'POS': 'PROPN'}]
#                ]

#     #checking the best match name and pulling it out
#     results = []
#     for pattern in patterns:
#       matcher.add('NAME', [pattern])

#       matches = matcher(nlp_text)

#       for match_id, start, end in matches:
#           span = nlp_text[start:end]
#           if not span.text.lower() in avoid_words:
#             results.append(span.text)
#             break
    
#     for result in results:
#       if (not any(element in avoid_words for element in result.split(' ')) ) and len(result)>1:
#         return result
          
#     return ""

In [ ]:
import spacy
from spacy.matcher import Matcher
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import words

def extract_name_using_Pronoun(content):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    ignore_words = ["curriculum" , "vitae" , "vita" , "post" , "applied" , "for" , "address" , "resume" , "engineering" , "engineer",
                "mechanical" , "address" , "mobile" , "contact" , "phone" , "temporary" , "street" , "resume", "computer" ,
                "software" , "personal" , "information" , "name" , "date" , "box" , "work" , "professional", "near" ,
                "objective" , "general" , "skills" , "skill" , "officer" , "administration" , "administrator" , "career", "position",
                "technology" , "IT", "qualification" , "academic", "experience", "department" , "email" , "e-mail" , "curriculam", 
                "viate" , "education" , "location"]

    new_content = removing_extra(content)
    # new_content = text_preprocessing(new_content)
    nlp_text = nlp(new_content)
    punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        foundwords = span.text.lower().split()
        found = 0
        for word in foundwords:
          if word in ignore_words:
            found = 1
            break
        if not(found):
          token_words = word_tokenize(span.text)
          flag = 0
          for word in token_words:
            if word in punctuations:
              flag = 1 
              break
          if flag == 0:
            return span.text
    return ""

In [ ]:
# import itertools

# def extract_name(text):
  
#   #Find names in Block letters
#   names = re.findall('[A-Z]+\.? ?[A-Z]+[\n|,| |\.]', text)

#   #Find names in capitalized letters  ([A-Z][a-z]+?\.?( ?[A-Z][a-z]+?){0,5} ?[A-Z][a-z]+?[\n| |,]) 
#   names_2 = re.findall('([A-Z][a-z]+?\.? ?( ?[A-Z][a-z]*?){0,5}\.? ?[A-Z][a-z]*?[\n| |,|.])', text)
#   names_2 = list(itertools.chain(*names_2))

#   #list of words to be avoided while parsing
#   avoid_words = ['resume','résumé','cv', 'curriculum','marketing', 'curriculum vitae','vitae', 'curriculum vita',
#                  'vita', 'name', 'pg', 'india', 'curriculam vitae', 'business development', 'cast', 'coc,','uae',
#                  'address', 'email', 'email address', 'sales', 'u.p', 'product development', 'personal statement',
#                  'full Name','street address', 'mobile', 'phone', 'number', 'phone number', 'mobile number', 'product',
#                  'agarwal', 'inter', 'secondary', 'steels', 'element', 'engineering','system', 'school', 'operating',
#                  'acknowledgement', 'assimilated', 'collaborated', 'collaboration','collaborations', 'india', 'isf,',
#                  'professional summary', 'professional', 'title', 'job title', 'job' 'technological', 'high', 'college',
#                  'top', 'device', 'launch', 'ams', 'crm', 'P Technological', 'project engineer', 'cgpa', 'gpa',
#                  'personal profile', 'years', 'lv', 'career', 'career profile','lv', 'curriculum viate', 'civil',
#                  'post applied', 'applied', 'post', 'for', 'electrical engineer', 'ps', 'district', 'ilets',
#                  'career objective.', 'it', 'carrier objective', 'electrical project handling', 'mig,', 'hvac',
#                  'mdu', 'linkedin', 'facebook', 'behance', 'youtube', 'github', 'in', 'portfolio','total experience',
#                  'pbd','working','xcx','wwd','mba','dtdc','cf ril', 'fi', 'ef', 'dd', 'project management',
#                  'engineer', 'apj', 'course university', 'university', 'ltd.', 'educational', 'n.y.', 's.s.',
#                  'information technology', 'uk', 'usa', 'uae','pakistan','china','canada','saudia','arabia',
#                  'educational background', 'details', 'personal details', 'clout', 'cloud', 'inditex', 'vy.', 'instrument',
#                  'instrument super', 'bca', 'u.p.', 'qa', 'gis', 'gre', 'aicte', 'professional and', 'education background',
#                  'tx', 'pvc', 'acadmic qualification', 'apply for merchandiser', 'education background','bechelor',
#                  'beardsell pvt.', 'class', 'fi ef', 'uae,', 'of', 'b.s.', 'project', ''
#                  'contributed', 'participated', 'participations', 'partnered', 'partnerships', 'volunteer work',
#                  'volunteer', 'side activities', 'projects', 'education', 'experience', 'work', 'work experience',
#                  'skills', 'interests', 'biography', 'bio', 'about me', 'about', 'info', 'contact', 'contact info',
#                  'reference', 'reference will be established upon request', 'certificates', 'additional certificates',
#                  'courses', 'student', 'online courses', 'short courses', 'languages', 'internships', 'organizations',
#                  'organization', 'matriculation', 'intermediate', 'pre-engineering', 'bachelors', 'masters',
#                  'bscs', 'bachelor of science', 'btech', 'btech', 'bachelor of technology', 'bs-cs,', 'sslc',
#                  'bse', 'bachelor of commerce', 'bcom', 'b.com', 'information', 'personal information',
#                  'info','achievements', 'personal projects', 'personal skills', 'summary', 'courses and certificates:',
#                  'profile', 'hobbies & interests', 'hobbies and interests', 'hobbies', 'career objective',
#                  'objective', 'other skills', 'academic', 'academics', 'academic qualification', 'references',
#                  'personal', 'cnic', 'dob', 'date of birth', 'work history','computer skills', 'referenc',
#                  'economics and finance', 'top skills', 'personal email', 'contact no', 'contact no.', 
#                  'cnic no', 'cnic no.', 'religion', 'domicile', 'a levels', 'o levels', 'working links',
#                  'links', 'working experience', 'awards and achievements', 'awards', 'soft skills', 
#                  'volunteer experience', 'conferences and courses', 'conferences', 'educational qualification',
#                  'educational qualifications', 'value addition', 'declaration', 'brief employment history',
#                  'extra activities', 'extra', 'skills and responsibilities', 'duties and responsibilities',
#                  'skills and abilities', 'area of expertise', 'technical skills', 'current address', 
#                  'permanent address', 'passport no', 'expiry date', 'career summary', 'email id', 'phone no',
#                  'contact no', 'contact no.', 'phone no.', 'education qualification', 'python', 'js6', 'ansys'
#                  ]

#   #variable to limit iteration
#   counter = 0
#   #Checks whether an avoid word is present and skips over it
#   for element in names:
#     if counter==3:
#       break
#     if not element.lower().replace('\n','').strip() in avoid_words:
#       return element
#     counter+=1
  
#   counter=0
#   for element in names_2:
#     if counter==4:
#       break
#     if not element.lower().replace('\n','').strip() in avoid_words:
#       return element
#     counter+=1
    
  
#   return ''

In [ ]:
import nltk
from nltk import word_tokenize

def extract_bigrams(text):
  #Create Bigrams
  tokens = word_tokenize(text)
  bigrams = list(nltk.bigrams(tokens))
  return bigrams

def form_bigrams(bigrams):
  #Forming possible combinations of first name and last name
  listOfPossible = []
  for word1,word2 in bigrams:
    name = str(word1) + " " + str(word2)
    listOfPossible.append(name)

  return listOfPossible

In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fuzzywuzzy import process
import pandas as pd

def extract_name(possibleNames , emailID , content):

  """Remove matches with punctuations and avoided words to get best results"""
  
  ignore_words = ["curriculum" , "vitae" , "vita" , "post" , "applied" , "for" , "address" , "resume" , "engineering" , "engineer",
                "mechanical" , "address" , "mobile" , "contact" , "phone" , "temporary" , "street" , "resume", "computer" ,
                "software" , "personal" , "information" , "name" , "date" , "box" , "work" , "professional", "near" ,
                "objective" , "general" , "skills" , "skill" , "officer" , "administration" , "administrator" , "career", "position",
                "technology" , "IT", "qualification" , "academic", "experience", "department" , "email" , "e-mail" , "curriculam", 
                "viate" , "education" , "location"]

  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
  if emailID == None or emailID == '' or len(emailID) < 1:
    return extract_name_using_Pronoun(content)

  if (type(emailID) == list):
    index = emailID[0].find('@')
    extractedEmail = emailID[0][0:index]
  else:
    index = emailID.find('@')
    extractedEmail = emailID[0:index]
     
  #Finding edit distance between two 
  
  matches = process.extract(str(extractedEmail) , possibleNames , limit = len(possibleNames))
  refinedList = []
  
  for match in matches:
    foundwords = match[0].lower().split()
    found = 0
    for word in foundwords:
      if word in ignore_words:
        found = 1
        break
    if not(found):
      token_words = word_tokenize(match[0])
      flag = 0
      for word in token_words:
        if word in punctuations:
          flag = 1 
          break
      if flag == 0:
        refinedList.append(match)
  print(refinedList)

# Extracting Phone Number

In [ ]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])
 
        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

# Extracting Email ID

In [ ]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

## Processing Email ID

In [ ]:
def process_emails(emailID):

  # Identifying the string slicing for gmail
  if type(emailID) == list:
    emailNum = 0
    for eml in emailID:
        for i in range(len(eml)):
          if eml[i] == '@':
            index1 = i
            break
        for j in range(index1 , len(eml)):
          if eml[j] == '.':
            index2 = j
            break
        if eml[index1 + 1:index2] == 'qmail' or eml[index1 + 1:index2] == 'gqmail':
          part1 = eml[0:index1 + 1]
          part2 = eml[index1 + 1 : index2]
          part3 = eml[index2:len(eml)]
          eml = part1 + 'gmail' + part3
        emailID[emailNum] = eml
        emailNum =  emailNum + 1
  else:
    for i in range(len(emailID)):
      if emailID[i] == '@':
        index1 = i
        break
    for j in range(index1 , len(emailID)):
      if emailID[j] == '.':
        index2 = j
        break
    part1 = emailID[0:index1 + 1]
    part2 = emailID[index1 + 1 : index2]
    part3 = emailID[index2:len(emailID)]
    emailID = part1 + 'gmail' + part3

# Main Program

In [ ]:
import csv
import glob

with open('output.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    for i in range(10):
        IMG_PATH = '/content/drive/MyDrive/sampleresumes/' + str(i+21)
        
        text = ""
        
        # Img Pre Processing
              
        img = cv2.imread(IMG_PATH + '/page0.jpeg' )

        img = ZoomImage(img , 3)
        img = GrayScaleImage(img)
        img = CropImage(img)
        img = InvertImage(img)

        cv2.imwrite(IMG_PATH + '/0.png', img)
        img = cv2.imread(IMG_PATH + '/0.png')

        # boundedImg = CreateBoundingBox(img)
        # cv2.imwrite(IMG_PATH + '/a.png', boundedImg)
        
        # img = cv2.imread(IMG_PATH + '/0.png')

        # NLP Part

        text = ExtractText(img)
        # print(text)      
        # personName = extract_name(text.title())
        contactNum = extract_phone_number(text.lower())
        emailID = extract_emails(text.lower())
        bigrams = extract_bigrams(text)
        possibleNames = form_bigrams(bigrams)
        process_emails(emailID)
        print(emailID)
        personName = extract_name(possibleNames , emailID , text)

        row = []
        row.append(personName)
        row.append(contactNum)
        row.append(emailID)
        print(row)
        print("\n")
        writer.writerow(row)
        break
    f.close()
        

['e-mail-kamilahmed502@gmail.com']
[('8392850914 E-Mail-kamilahmed502', 90), ('in 2012', 49), ('Affiliated by', 46), ('Kamil Vill-', 45), ('Roorkee Affiliated', 42), ('Basic Knowledge', 40), ('PLC SCADA', 40), ('PLC SCADA', 40), ('a highly', 39), ('from UK', 39), ('in 2014', 39), ('from UK', 39), ('a job', 36), ('where |', 36), ('and knowledge', 36), ('Diploma in', 36), ('pursue a', 34), ('utilize my', 34), ('School Passed', 34), ('» Attended', 34), ('of Vocational', 34), ('can utilize', 32), ('Passed with', 32), ('PO- Salempur', 31), ('job in', 30), ('| can', 30), ('of IMS', 30), ('Knowledge of', 30), ('one month', 30), ('Zenus Infoted', 30), ('Vill- Safarpur', 29), ('Intermediate Passed', 29), ('knowledge efficiently', 28), ('2012 TECHNICAL', 28), ('years Diploma', 28), ('IMS Institute', 28), ('Attended one', 28), ('training in', 28), ('in P.T.C.U.L', 28), ('Automation PLC', 28), ('SCADA Course', 28), ('training from', 28), ('of Industrial', 28), ('Automation PLC', 28), ('SCADA Cours